Balogh Szilard, Bajan Ramona-Maria, Popa Sebastian

Exercise 2

In [154]:
import time
import numpy as np
from math import comb
from itertools import combinations

In [155]:
def generate_binary_matrix(M, N, p):
    return np.random.binomial(1, p, (M, N))

In [156]:
def create_candidates(frequent_itemsets, k):
    candidates = set()
    itemsets_list = list(map(sorted, frequent_itemsets))  # Sort items to ensure the order

    for i in range(len(itemsets_list)):
        for j in range(i + 1, len(itemsets_list)):
            # Check if the first k-2 items are the same
            if itemsets_list[i][:-1] == itemsets_list[j][:-1]:
                # Join the two itemsets
                candidate_set = set(itemsets_list[i]).union(set(itemsets_list[j]))
                if len(candidate_set) == k:
                    candidates.add(tuple(sorted(candidate_set)))
    
    return candidates

In [157]:
class HashTreeNode:
    def __init__(self, is_leaf=True, bucket_size=3):
        self.is_leaf = is_leaf
        self.buckets = {} if not is_leaf else []
        self.bucket_size = bucket_size if is_leaf else None
        self.support = {}  # only meaningful in leaf nodes

class HashTree:
    def __init__(self, k, hash_func=lambda x: x % 3, bucket_size=3):
        self.root = HashTreeNode()
        self.k = k
        self.bucket_size = bucket_size
        self.hash_func = hash_func

    def insert(self, tuple_value):
        self._insert_recursive(self.root, tuple_value, level=1)

    def _insert_recursive(self, node, tuple_value, level):
        if node.is_leaf:
            if tuple_value not in node.support:
                if len(node.buckets) < node.bucket_size or level > self.k:
                    node.buckets.append(tuple_value)
                    node.support[tuple_value] = 0
                else:
                    self._split_node(node, level)
                    self._insert_recursive(node, tuple_value, level)
            return

        key = self.hash_func(tuple_value[level - 1])
        if key not in node.buckets:
            node.buckets[key] = HashTreeNode()
        self._insert_recursive(node.buckets[key], tuple_value, level + 1)

    def _split_node(self, node, level):
        new_node = HashTreeNode(is_leaf=False)
        for t in node.buckets:
            key = self.hash_func(t[level - 1])
            if key not in new_node.buckets:
                new_node.buckets[key] = HashTreeNode()
            new_node.buckets[key].buckets.append(t)
            new_node.buckets[key].support[t] = node.support[t]
        node.is_leaf = False
        node.buckets = new_node.buckets
        node.support = None

    def insert_candidates(self, candidates):
        for candidate in candidates:
            self.insert(candidate)

    def count_occurrences(self, transaction):
        for subset in combinations(transaction, self.k):
            self._count_single_subset(self.root, subset, level=1)

    def _count_single_subset(self, node, subset, level):
        if node.is_leaf:
            if subset in node.support:
                node.support[subset] += 1
            return
        key = self.hash_func(subset[level - 1])
        if key in node.buckets:
            self._count_single_subset(node.buckets[key], subset, level + 1)

    def get_subsets(transaction, k):
        if len(transaction) < k:
            return []
        return combinations(transaction, k)  

    def get_supported_candidates(self, min_support):
        result = []

        def collect(node):
            if node.is_leaf:
                result.extend([item for item, count in node.support.items() if count >= min_support])
            else:
                for child in node.buckets.values():
                    collect(child)

        collect(self.root)
        return result

    def get_leaf_statistics(self):
        leaf_count = 0
        leaf_sizes = []

        def count(node):
            nonlocal leaf_count
            if node.is_leaf:
                leaf_count += 1
                leaf_sizes.append(len(node.buckets))
            else:
                for child in node.buckets.values():
                    count(child)

        count(self.root)
        return leaf_count, leaf_sizes

In [158]:
def apriori_using_indices(matrix: np.ndarray, min_support, verbose=False):
    num_transactions, num_items = matrix.shape

    # Step 1: Count the support for each individual item (using indices)
    support = matrix.sum(axis=0)
    single_items = {index: support[index] for index in range(num_items)}

    if verbose:
        print(f"1-itemsets: {single_items}")

    # Step 2: Remove items below min_support
    single_items = {k: v for k, v in single_items.items() if v >= min_support}

    # Convert frequent single items to a list of sets containing indices
    frequent_itemsets = [tuple([index]) for index in single_items.keys()]

    if verbose:
        print(f"1-itemsets above min_support: {len(frequent_itemsets)}")

    all_frequent_itemsets = []
    all_frequent_itemsets.extend(frequent_itemsets)

    k = 2
    while frequent_itemsets:
        # Step 3: Generate candidate itemsets of length k using indices
        candidates = create_candidates(frequent_itemsets, k)

        # Step 4: Count support of candidates using the binary matrix
        candidate_support_counts = {candidate: 0 for candidate in candidates}

        for candidate in candidates:
            # Create a mask for rows that contain the candidate itemset
            indices = list(candidate)
            mask = matrix[:, indices].all(axis=1)  # All items must be present

            # Count how many transactions contain the candidate itemset
            candidate_support_counts[candidate] = np.sum(mask)

        if verbose:
            print(f"{k}-itemsets candidates: {len(candidate_support_counts)}")

        # Step 5: Prune non-frequent itemsets
        frequent_itemsets = [c for c, count in candidate_support_counts.items() if count >= min_support]

        if verbose:
            print(f"{k}-itemsets above min_support: {len(frequent_itemsets)}")
        all_frequent_itemsets.extend(frequent_itemsets)

        # Step 6: Move to the next level
        k += 1

    return all_frequent_itemsets

In [159]:
def apriori_with_hash_tree(matrix, min_support, hash_func, verbose=False):
    num_transactions, num_items = matrix.shape

    support = matrix.sum(axis=0)
    single_items = {index: support[index] for index in range(num_items)}
    single_items = {k: v for k, v in single_items.items() if v >= min_support}
    frequent_itemsets = [tuple([i]) for i in single_items.keys()]
    all_frequent_itemsets = frequent_itemsets[:]

    k = 2
    while frequent_itemsets:
        candidates = create_candidates(frequent_itemsets, k)
        tree = HashTree(k, hash_func)
        tree.insert_candidates(candidates)

        for row in matrix:
            transaction = tuple(i for i, bit in enumerate(row) if bit == 1)
            if len(transaction) >= k:
                tree.count_occurrences(transaction)

        frequent_itemsets = tree.get_supported_candidates(min_support)
        all_frequent_itemsets.extend(frequent_itemsets)

        if verbose:
            print(f"{k}-itemsets above min_support: {len(frequent_itemsets)}")
        k += 1

    return all_frequent_itemsets

In [160]:
def total_search_space(n):
    return sum(comb(n, k) for k in range(1, n + 1))

In [161]:
def evaluate(matrix, min_support, alg, hash = None, verbose=False):
    print(f"Evaluate algorithm {alg.__name__}")
    if not hash:
        start = time.time()
    if hash:
        print("Hash function used: " + str(hash.__name__))
        start = time.time()
        frequent_itemsets = alg(matrix, min_support, hash, verbose)
    else:
        frequent_itemsets = alg(matrix, min_support, verbose)
    end = time.time()
    print(f"Time: {end - start} seconds")
    print(f"Number of frequent itemsets: {len(frequent_itemsets)}")
    print()

In [162]:
def apriori_with_hash_tree_analysis(matrix, min_support, hash_func, verbose=False):
    num_transactions, num_items = matrix.shape
    support = matrix.sum(axis=0)
    single_items = {index: support[index] for index in range(num_items)}
    single_items = {k: v for k, v in single_items.items() if v >= min_support}
    frequent_itemsets = [tuple([i]) for i in single_items.keys()]
    all_frequent_itemsets = frequent_itemsets[:]

    k = 2
    total_candidates = 0
    total_false_alarms = 0
    while frequent_itemsets:
        candidates = create_candidates(frequent_itemsets, k)
        total_candidates += len(candidates)
        tree = HashTree(k, hash_func=hash_func)
        tree.insert_candidates(candidates)

        for row in matrix:
            transaction = tuple(i for i, bit in enumerate(row) if bit == 1)
            if len(transaction) >= k:
                tree.count_occurrences(transaction)

        frequent_itemsets = tree.get_supported_candidates(min_support)
        total_false_alarms += len(candidates) - len(frequent_itemsets)
        all_frequent_itemsets.extend(frequent_itemsets)

        if verbose:
            print(f"{k}-itemsets: {len(frequent_itemsets)}")
        k += 1

    if 'tree' in locals():
        leaf_count, leaf_sizes = tree.get_leaf_statistics()
    else:
        leaf_count, leaf_sizes = 0, []
    search_space = total_search_space(num_items)
    pruning_rate = (1 - total_candidates / search_space) * 100
    false_alarm_rate = (total_false_alarms / total_candidates) * 100 if total_candidates else 0
    freq_percent = (len(all_frequent_itemsets) / search_space) * 100

    return {
        'frequent_itemsets': all_frequent_itemsets,
        'leaf_count': leaf_count,
        'leaf_sizes': leaf_sizes,
        'search_space': search_space,
        'total_candidates': total_candidates,
        'pruning_rate': pruning_rate,
        'false_alarm_rate': false_alarm_rate,
        'frequent_percent': freq_percent
    }

def evaluate_analysis(matrix, min_support, hash_func):
    start = time.time()
    results = apriori_with_hash_tree_analysis(matrix, min_support, hash_func, verbose=True)
    end = time.time()

    print("\nEvaluation Summary:")
    print(f"Time taken: {end - start:.4f} seconds")
    print(f"Number of frequent itemsets: {len(results['frequent_itemsets'])}")
    print(f"Frequent % of total search space: {results['frequent_percent']:.4f}%")
    print(f"Pruning rate: {results['pruning_rate']:.2f}%")
    print(f"False alarm rate: {results['false_alarm_rate']:.2f}%")
    print(f"Number of leaf nodes: {results['leaf_count']}")
    print(f"Average leaf size: {np.mean(results['leaf_sizes']):.2f}")

In [163]:
def simple_hash_func(x):
    return x % 1

def hash_5(x):
    return hash(x) % 5

def knuth_hash_func(x):
    return (x * 2654435761) % 2**32 % 7

matrix = generate_binary_matrix(1000, 30, 0.5)
support_ratio = 0.2
min_support = support_ratio * matrix.shape[0]
evaluate(matrix, min_support, apriori_using_indices)
evaluate(matrix, min_support, apriori_with_hash_tree, simple_hash_func)
evaluate(matrix, min_support, apriori_with_hash_tree, hash_5)
evaluate(matrix, min_support, apriori_with_hash_tree, knuth_hash_func)

Evaluate algorithm apriori_using_indices
Time: 0.10713887214660645 seconds
Number of frequent itemsets: 465

Evaluate algorithm apriori_with_hash_tree
Hash function used: simple_hash_func
Time: 0.4795854091644287 seconds
Number of frequent itemsets: 465

Evaluate algorithm apriori_with_hash_tree
Hash function used: hash_5
Time: 0.5210635662078857 seconds
Number of frequent itemsets: 465

Evaluate algorithm apriori_with_hash_tree
Hash function used: knuth_hash_func
Time: 0.7660059928894043 seconds
Number of frequent itemsets: 465



In [164]:
evaluate_analysis(matrix, min_support, hash_func=simple_hash_func)
evaluate_analysis(matrix, min_support, hash_func=hash_5)
evaluate_analysis(matrix, min_support, hash_func=knuth_hash_func)

2-itemsets: 435
3-itemsets: 0

Evaluation Summary:
Time taken: 0.4942 seconds
Number of frequent itemsets: 465
Frequent % of total search space: 0.0000%
Pruning rate: 100.00%
False alarm rate: 90.32%
Number of leaf nodes: 1
Average leaf size: 4060.00
2-itemsets: 435
3-itemsets: 0

Evaluation Summary:
Time taken: 0.5017 seconds
Number of frequent itemsets: 465
Frequent % of total search space: 0.0000%
Pruning rate: 100.00%
False alarm rate: 90.32%
Number of leaf nodes: 125
Average leaf size: 32.48
2-itemsets: 435
3-itemsets: 0

Evaluation Summary:
Time taken: 0.7386 seconds
Number of frequent itemsets: 465
Frequent % of total search space: 0.0000%
Pruning rate: 100.00%
False alarm rate: 90.32%
Number of leaf nodes: 197
Average leaf size: 20.61


We have observed that even though in theory the hash approach is quicker than the "Brute force" approach, in Python, the apriori_using_indices usually finishes quicker than the hash tree. On the one hand, the reason is that apriori_using_indices uses the vectorization from numpy, which is very fast (thanks to it being implemented in C and not in Python): mask = matrix[:, indices].all(axis=1), on the other hand, the hash tree approach suffers from the fact that it has to construct a new hash tree for each value of 'k'. For instance, with a matrix consisting of 500 transactions, each of which randomly can have at most 10 elements, with the probability 50% of containing an element, we got the following results: for the "normal" approach, the execution time was: 0.0051 seconds, for the hash tree approach with the hashing function h(x) = x % 1, the time was 0.031 seconds, using the hash function h(x) = hash(x) % 5 we obtained 0.021 seconds and for the knuth_hash_func we obtained 0.016 seconds. We have also noticed that the hash function we choose greately influences how the tree is constructed and implicitly, the number and size of leaves. For instance, with the example we have desribed above, for our first hash function, we obtained a single leaf with 120 itemsets. On the other hand, the second hash function gave us 79 leaves with an average leaf size of 1.52, meanwhile the third gave us 47 leaves with an average leaf size of 2.55. On the other hand, the hash function does not influence the false alarm rate, the pruning rate and the frequent % of the total search space. For the mentioned example, we have obtained a frequency percent of the total search space of 5.37%, a pruning rate of 83.87% and a false alarm rate of 72.73%. We have also noticed that with the increase of the number of columns comes the increase of the pruning rate and the false alarm rate and the decrease of the frequency of the itemsets from the total search space.

Exercise 3

In [165]:
import numpy as np
import pandas as pd

def load_transaction_dataset(filepath):
    with open(filepath, 'r') as f:
        lines = f.readlines()
    transactions = [list(map(int, line.strip().split())) for line in lines if line.strip()]
    return transactions

def transactions_to_matrix(transactions):
    all_items = sorted(set(item for tx in transactions for item in tx))
    item_index = {item: idx for idx, item in enumerate(all_items)}
    matrix = np.zeros((len(transactions), len(all_items)), dtype=int)
    for i, tx in enumerate(transactions):
        for item in tx:
            matrix[i][item_index[item]] = 1
    return matrix, item_index

In [166]:
from mlxtend.frequent_patterns import apriori as mlxtend_apriori

def benchmark_dataset(filepath, min_support_ratio=0.01):
    print(f"\n===== Benchmark for: {filepath} =====")
    transactions = load_transaction_dataset(filepath)
    matrix, item_index = transactions_to_matrix(transactions)
    min_support = int(min_support_ratio * len(matrix))
    
    print("→ Custom Apriori with Hash Tree:")
    start = time.time()
    custom_result = apriori_with_hash_tree(matrix, min_support, hash_5)
    custom_time = time.time() - start
    print(f"Time: {custom_time:.4f}s | Frequent itemsets: {len(custom_result)}")

    print("→ mlxtend Apriori:")
    df = pd.DataFrame(matrix, columns=[f'item_{i}' for i in range(matrix.shape[1])]).astype(bool)
    start = time.time()
    mlxtend_result = mlxtend_apriori(df, min_support=min_support_ratio, use_colnames=True)
    mlxtend_time = time.time() - start
    print(f"Time: {mlxtend_time:.4f}s | Frequent itemsets: {len(mlxtend_result)}")

    return {
        'dataset': filepath,
        'custom_time': custom_time,
        'mlxtend_time': mlxtend_time,
        'custom_count': len(custom_result),
        'mlxtend_count': len(mlxtend_result)
    }

results = []
for file in ['chess.dat.txt', 'mushroom.dat.txt', 'retail.dat.txt']:
    results.append(benchmark_dataset(file, min_support_ratio=0.99))


===== Benchmark for: chess.dat.txt =====
→ Custom Apriori with Hash Tree:
Time: 32.4214s | Frequent itemsets: 9
→ mlxtend Apriori:
Time: 0.0028s | Frequent itemsets: 9

===== Benchmark for: mushroom.dat.txt =====
→ Custom Apriori with Hash Tree:
Time: 0.4564s | Frequent itemsets: 1
→ mlxtend Apriori:
Time: 0.0031s | Frequent itemsets: 1

===== Benchmark for: retail.dat.txt =====
→ Custom Apriori with Hash Tree:
Time: 3.9010s | Frequent itemsets: 0
→ mlxtend Apriori:
Time: 1.5955s | Frequent itemsets: 0


With our benchmarking, we have noticed that the mlxtend implementation of apriori is considerably more efficient than our implementation. For instance, using the chess dataset, it took 31.89 seconds for our implementation to find 9 frequent datasets, meanwhile the same thing only took 0.0044 seconds for the mlxtend algorithm. Similarly, in the mushroom dataset, it took our algorithm 0.45 seconds to find 1 frequent itemset, which was found in 0.0031 seconds by mlxtend. In the retail dataset, where there were no frequent itemsets found due to the high min_support_ration, it took our algorithm 3.90 seconds to finish and it took 1.59 seconds for the mlxtend algorithm.